# Build graph from preprocessed file
With a tabulated count:san file

## Read in packages, functions, and file

In [16]:
import sys
import networkx as nx
import matplotlib as plt
import numpy as np
import re
import chess
import subprocess
import timeit
import pickle
import stockfish
from stockfish import Stockfish
import chess
import chess.engine
import warnings
warnings.filterwarnings('ignore')

engine = chess.engine.SimpleEngine.popen_uci("../stockfish-10-64")
stockfish = Stockfish("../stockfish-10-64")

In [17]:
def varWeighted(scores, counts):
    if (len(counts)<2):
        return None
    else:
        weightedMean = sum([a*b for a,b in zip(scores,counts)])/sum(counts)
        scoreDiffs = [(score - weightedMean)**2 for score in scores]
        V1 = sum(counts)
        V2 = sum([count**2 for count in counts])
        var = (V1/(V1**2-V2)) * sum([a*b for a,b in zip(counts,scoreDiffs)])
        return var
    
def get_score(san):
    board = chess.Board()
    for move in san:
        board.push_san(move)
    currFen = board.fen()
    score = engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE)
    if('#' in str(score['score'])):
        if('-' in str(score['score'])):
            return -39765
        else:
            return 39765
    else:
        score = int(str(score["score"]))
    return score

def san_to_fen(san):
    board = chess.Board()
    for move in san:
        board.push_san(move)
    currFen = board.fen()
    return currFen

def get_node_sd(node):
    scores = []
    counts = []
    try:
        for neighbor in list(g.neighbors(node)):
            scores.append(nx.get_node_attributes(g, 'score')[neighbor])
            counts.append(nx.get_node_attributes(g, 'movelistCount')[neighbor][node])
        return node, np.sqrt(varWeighted(scores,counts))
    except:
        return node, None

In [18]:
game_file = "../tab-trunc.txt"

with open(game_file, "r") as file_in:
    mainlineList = []
    for line in file_in:
        mainlineList.append(line)

#extract each string of moves
stringList = []
for mainLine in mainlineList:
    stringList.append(mainLine.split())

# stringList.sort(key = len) 


In [19]:
print(len(stringList))

692287


## Build and pickle graph

In [20]:
cnt = 0
start = timeit.default_timer()

#initialize graph with root node
g = nx.DiGraph()
g.add_node('root', score = 0, movelist = 'root')

for s in stringList:
    whiteMove = 1
    if(cnt%10000 == 0):
        print("strings processed:", cnt, "TIME:", timeit.default_timer()-start)
    count = int(s[0])
    if(count >= 50):
        board = chess.Board()
        parentFen = 'root'
        for move in s[1:]:
            try:
                san = s[1:s.index(move)+1]
                board.push_san(move)
                currFen = board.fen()
                if(currFen not in g.nodes):
                    board = chess.Board(currFen)
                    if('#' not in move):
                        score = engine.analyse(board, chess.engine.Limit(time=0.05))
                        if('#' in str(score['score'])):
                            if('-' in str(score['score'])):
                                score =  -39765
                            else:
                                score =  39765
                        else:
                            score = int(str(score["score"])) * whiteMove
    #                         stockfish.set_fen_position(currFen)
    #                         score = stockfish.get_evaluation()['value']
                    else:
                        score = -39765 * whiteMove
                    g.add_node(currFen,
                               score = score,
                               count = count,
                               san = san,
                               movelistCount = {parentFen:count})
                    g.add_edge(parentFen, currFen)
                else:
                    if(parentFen in nx.get_node_attributes(g, 'movelistCount')[currFen]):
                        nx.get_node_attributes(g, 'movelistCount')[currFen][parentFen] += count #increase parent count
                    else:
                        nx.get_node_attributes(g, 'movelistCount')[currFen][parentFen] = count #add parent and count
                        g.add_edge(parentFen, currFen) #add edge from parentFen to currFen
                    g.nodes[currFen]['count'] = nx.get_node_attributes(g, 'count')[currFen] + count #increase count
    #                 whiteMove *= -1
                parentFen = currFen
            except Exception as e:
                print(e)
                print(sys.exc_info()[0])
                print("Move",s)
                stockfish.set_fen_position(currFen)
                print(stockfish.get_evaluation()['value'])
                break
        cnt+=1
    else:
        cnt+=1
            
stop = timeit.default_timer()
print('Time: ', stop - start)
print("totalNodes:",len(g))

strings processed: 0 TIME: 0.011848312000438455
strings processed: 10000 TIME: 33.230453473000125
strings processed: 20000 TIME: 35.83111786399968
strings processed: 30000 TIME: 40.355436019999615
strings processed: 40000 TIME: 41.9361902820001
strings processed: 50000 TIME: 45.41323559800003
strings processed: 60000 TIME: 50.10824482800035
strings processed: 70000 TIME: 54.40848151600039
strings processed: 80000 TIME: 57.18194516800031
strings processed: 90000 TIME: 61.01557742899968
strings processed: 100000 TIME: 63.567637973000274
strings processed: 110000 TIME: 70.16195504799998
strings processed: 120000 TIME: 74.25952541900006
strings processed: 130000 TIME: 79.53666932099986
strings processed: 140000 TIME: 84.33882603799975
strings processed: 150000 TIME: 87.98654814699967
strings processed: 160000 TIME: 92.85023949700008
strings processed: 170000 TIME: 100.75500999999986
strings processed: 180000 TIME: 107.18212666399995
strings processed: 190000 TIME: 111.45687470200028
string

In [21]:
g.nodes()['rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2']

{'score': 44,
 'count': 11948,
 'san': ['d4', 'Nf6'],
 'movelistCount': {'rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1': 11948}}

In [12]:
#nx.write_gpickle(g,"test.gpickle")

In [22]:
g = nx.read_gpickle('full690k.gpickle')

10346

In [27]:
g.nodes()[san_to_fen(['c4', 'Nf6', 'Nf3'])]

{'score': -23,
 'count': 50,
 'san': ['c4', 'Nf6', 'Nf3'],
 'movelistCount': {'rnbqkb1r/pppppppp/5n2/8/2P5/8/PP1PPPPP/RNBQKBNR w KQkq - 1 2': 50}}

## Test variance calcuilations

In [14]:
start = timeit.default_timer()

board = chess.Board()
board.push_san('d4') #push the move to the board
board.push_san('Nf6') #push the move to the board
currFen = board.fen()
print(currFen)
scores = []
counts = []
for neighbor in list(g.neighbors(currFen)):
    scores.append(nx.get_node_attributes(g, 'score')[neighbor])
    counts.append(nx.get_node_attributes(g, 'count')[neighbor])
    
print("sd:",np.sqrt(varWeighted(scores, counts)))

stop = timeit.default_timer()
print('Time: ', stop - start) 

rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2
sd: 39.42412035207993
Time:  0.19528749800019796


In [15]:
gamesDict = {"root" : ['e4', 'c5'],
"safe" : ['e4', 'c5', 'c3'],
"risky" : ['e4', 'c5', 'd4'],
"root2" : ['e4', 'e6', 'd4', 'd5'],
"popular2.0" : ['e4', 'e6', 'd4', 'd5', 'Nc3'],
"popular2.1" : ['e4', 'e6', 'd4', 'd5', 'Nd2'],
"safe2" : ['e4', 'e6', 'd4', 'd5', 'exd5'],
"risky2" : ['e4', 'e6', 'd4', 'd5', 'e5'],
"Ruy" : ['e4', 'e5','Nf3', 'Nc6','Bb5'],
"Vienna" : ['e4' ,'e5','d4','exd4','c3']}

for key, game in gamesDict.items():
    currFen = san_to_fen(game)
    scores = []
    counts = []
    try:
        for neighbor in list(g.neighbors(currFen)):
            scores.append(nx.get_node_attributes(g, 'score')[neighbor])
            counts.append(nx.get_node_attributes(g, 'movelistCount')[neighbor][currFen])
        print('{} {} ||| SD :'.format(key, game), np.sqrt(varWeighted(scores,counts)), ", total Games:", sum(counts))
    except:
        break

    

root ['e4', 'c5'] ||| SD : 52.4178835913871 , total Games: 30613
safe ['e4', 'c5', 'c3'] ||| SD : 17.919670383656534 , total Games: 1224
risky ['e4', 'c5', 'd4'] ||| SD : 87.61629331075976 , total Games: 3702
root2 ['e4', 'e6', 'd4', 'd5'] ||| SD : 34.549936453466835 , total Games: 5868
popular2.0 ['e4', 'e6', 'd4', 'd5', 'Nc3'] ||| SD : 26.534029951240356 , total Games: 986
popular2.1 ['e4', 'e6', 'd4', 'd5', 'Nd2'] ||| SD : 29.42564695849045 , total Games: 677
safe2 ['e4', 'e6', 'd4', 'd5', 'exd5'] ||| SD : 79.12339016299454 , total Games: 639
risky2 ['e4', 'e6', 'd4', 'd5', 'e5'] ||| SD : 48.575727612339946 , total Games: 4571
Ruy ['e4', 'e5', 'Nf3', 'Nc6', 'Bb5'] ||| SD : 32.326789052452725 , total Games: 4814
Vienna ['e4', 'e5', 'd4', 'exd4', 'c3'] ||| SD : 43.9007496294731 , total Games: 753


In [ ]:
sd_dict = {}
for node in g.nodes():
    n, val = get_node_sd(node)
    sd_dict.update({n: val})
#     dict.update(Iterable_Sequence of key:value)
#     sd_list.append(get_node_sd(node))

sd_dict = {key:val for key, val in sd_dict.items() if val != None}

maxSD = max(sd_dict.items(), key=lambda x : x[1])
print('Max value in Dict: ', maxSD[1])
print('Key With Max value in Dict: ', maxSD[0])

In [ ]:
{k: v for k, v in sorted(sd_dict.items(), key=lambda item: item[1], reverse = True)}

In [ ]:
currFen =  'rnbqkbnr/p3pppp/8/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR b KQkq - 1 6'
scores = []
counts = []

for neighbor in list(g.neighbors(currFen)):
    print(neighbor, g.nodes()[neighbor])
    scores.append(nx.get_node_attributes(g, 'score')[neighbor])
    counts.append(nx.get_node_attributes(g, 'movelistCount')[neighbor][currFen])
print('SD :', np.sqrt(varWeighted(scores,counts)), ", total Games:", sum(counts))


## Ad hoc viz of moves and children with sf score

In [5]:
#Move ['32', 'f3', 'e5', 'g4', 'Qh4#']

board = chess.Board()
board.push_san('f3') #push the move to the board
board.push_san('e5') #push the move to the board
#board.push_san('g4') #push the move to the board
#board.push_san('Qh4#') #push the move to the board
currFen = board.fen()
print(currFen)
stockfish.set_fen_position(currFen)
score = stockfish.get_evaluation()['value']
print(score)

rnbqkbnr/pppp1ppp/8/4p3/8/5P2/PPPPP1PP/RNBQKBNR w KQkq - 0 2
-101


In [ ]:
board = chess.Board(currFen)
print(engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE))
board

In [ ]:
board = chess.Board(currFen)
print(engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE))
print("CURRFEN:")
print(board)
print("_"*50)
print("_"*50)
for neighbor in list(g.neighbors(currFen)):
    board = chess.Board(neighbor)
    print(engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE))
    print(board)
    print("_"*50)